<a href="https://colab.research.google.com/github/Alishaw99/Agents_pdf_assistant_phidta/blob/main/autogen_multiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogen

In [ ]:
!pip install pyautogen

In [ ]:
!pip install dask[dataframe]

In [ ]:
import autogen

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
config_list = [{'model': 'gpt-4o', 'api_key':OPENAI_API_KEY }]

In [ ]:
llm_config = {"config_list": config_list}

In [ ]:
gpt4o_config = {
    "cache_seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

In [ ]:
task = '''
  **Task**: As an architect, you are required to design a solution for the following business requriments:
  - Data storage for massive amount of IOT data
  - Real-time data analytics and machine learning pipeline
  -Scalability
  -Cost Optimizaiton
  -Regional paris in Europe, for disaster recovery
  -Tools for monitoring and observability
  -Time: 6 month

  Break down the problem using a Chain-of-Though approach. Ensure that your solution artictecture is
  following the best practices.
  '''

In [ ]:
  cloud_prompt= '''
  **Role**: you are an expert cloud architect. you need to develop architecture proposals
  using either cloud-specifice PaaS services, or cloud-agnostic ones.
  The final propsoal should consider all 3 main clouc providers: Azure, AWS and GCP, and provide
  a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
  architectures, and summarize your solutions for each cloue provider using a table for clarity.
  '''
  cloud_prompt += task

In [ ]:
oss_prompt = '''
**Role**: you are an expert on premises, open-source software architect. You need to develop
architecture proposal without considering cloud solution. only use open-source frameworks that are
popular and have lots of active contributors. At the end, briefly state the advantages of
open-source adoption, and summarize your solutions using a table for clarity.`
'''
oss_prompt += task

In [ ]:
lead_prompt = '''
**Role**: you are an lead architect tasked with managing a conversation between the cloud and the open-source Architects.
Each Architect will perform a task and respond with their results. You will critically review
those and also ask for, or point to, the disadvantages of thier solutions.
You will review each results, and choose the best solution in accordance with the business requirement and architecture
best practices. You will use any number of summary table to communicate your decisionexpert on premises, open-source software architect. You need to develop
architecture proposal without considering cloud solution. only use open-source frameworks that are
popular and have lots of active contributors. At the end, briefly state the advantages of
open-source adoption, and summarize your solutions using a table for clarity.`
'''
lead_prompt += task

In [ ]:
user_proxy = UserProxyAgent(
    name="supervisor",
    system_message = "A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",
)


In [ ]:
cloud_agent = AssistantAgent(
    name="cloud",
    system_message=cloud_prompt,
    llm_config={"config_list": config_list}
)

oss_agent = AssistantAgent(
    name="oss",
    system_message=oss_prompt,
    llm_config={"config_list": config_list}
)

lead_agent = AssistantAgent(
    name="lead",
    system_message=lead_prompt,
    llm_config={"config_list": config_list}
)


In [ ]:
def state_transition(last_speaker, groupchat):
  messages = groupchat.messages

  if last_speaker is user_proxy:
    return cloud_agent
  elif last_speaker is cloud_agent:
    return oss_agent
  elif last_speaker is oss_agent:
    return lead_agent
  elif last_speaker is lead_agent:
    # lead-> end
    return None

In [ ]:
LLMConfig = {"config_list": config_list}

In [ ]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, cloud_agent, oss_agent, lead_agent],
    messages=[],
    max_round=6,
    speaker_selection_method=state_transition,

)

In [ ]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on the AI agent for the business requirements"
)